<a href="https://colab.research.google.com/github/MahreenAthar/MLOps-Assignment-2/blob/BranchMahreen/Model_StockExchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import datetime
import os
import joblib
from flask import Flask, request, jsonify, render_template
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
import yfinance as yf

In [3]:
app = Flask(__name__, template_folder='/app')

# Loading the dataset

In [4]:
ticker = yf.Ticker("ZTL")
data = ticker.history(start=datetime.date(2020, 1, 1), end=datetime.date.today())
data.reset_index(inplace=True)

ZTL: No data found for this date range, symbol may be delisted


# Splitting Data
80% training, 20% testing

In [5]:
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

# Define input features 
Open, High, Low, and Volume

In [6]:
X_train = train_data[['Open', 'High', 'Low', 'Volume']]
X_test = test_data[['Open', 'High', 'Low', 'Volume']]